In [1]:
import os
import sys
sys.path.insert(1, os.path.join('..', 'src'))
import pickle
import numpy as np
import pandas as pd

In [2]:
from wuhan_functions import load_data_live, load_data_pkl, load_data_csv

## Load Live Data

In [46]:
def fetch_stat():
    """
    Return a DataFrame that represent death, confirmed, investigating and reported numbers respectively.
    
    There are two elements in the json that contain the required value: 
    "allBotWarsLatestFigures" and "allWarsLatestFiguresOverride".
    
    The values in allBotWarsLatestFigures are the record in history for every day.
    The values in allWarsLatestFiguresOverride are the live values.
    Whenenver an attribute in allWarsLatestFiguresOverride is not an empty string,
    it overrides the values in allBotWarsLatestFigures.
    
    """
    statnames = ['Death', 'Confirmed', 'Investigating', 'Reported']
    
    page = requests.get('https://wars.vote4.hk/page-data/cases/page-data.json')
    data = json.loads(page.content)
    print(json.dumps(data, indent=4, sort_keys=True))
    stats_yesterday = data['result']['data']['allBotWarsLatestFigures']['edges'][0]
    stats_live = data['result']['data']['allWarsLatestFiguresOverride']['edges'][0]
    
    res = {}
    
    for attr in ['death', 'confirmed', 'investigating', 'reported', 'discharged', 'ruled_out']:
        if stats_live['node'][attr] != "":
            res[attr] = stats_live['node'][attr]
        else:
            res[attr] = stats_yesterday['node'][attr]
    
    df = pd.DataFrame(data=res, index=[0])
    
    return df

In [47]:
fetch_stat()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyError: 'allBotWarsLatestFigures'

In [3]:
cases_df, high_risk_df, stats_df, hospital_awaiting_df = load_data_live()

KeyError: 'data'

In [48]:
from webscraper import fetch_highrisk

In [62]:
from lxml import html
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import json

In [63]:
df = fetch_highrisk()

In [64]:
flourish_df = df[['lat', 'lng', 'start_date', 'end_date', 'action_en']]

In [65]:
flourish_df = flourish_df[
    (flourish_df['lat'] != '') &
    (flourish_df['lng'] != '') &
    ~(flourish_df['start_date'].isna()) &
    ~(flourish_df['end_date'].isna())
]

In [71]:
flourish_df['start_date'] = flourish_df['start_date'].dt.strftime("%Y-%m-%dT%H:%M:%S.000Z")
flourish_df['end_date'] = flourish_df['end_date'].dt.strftime("%Y-%m-%dT%H:%M:%S.000Z")

In [72]:
flourish_df.shape

(945, 6)

In [73]:
flourish_df['Value'] = 1

In [75]:
flourish_df.to_excel('flourish_df.xlsx')

In [77]:
flourish_df['lat'].min()


'22.204486'

In [78]:
flourish_df['lat'].max()

'22.529605'

In [79]:
flourish_df['lng'].min()

'113.918693'

In [81]:
flourish_df['lng'].max()

'22.280605'

In [83]:
flourish_df['lng'].sort_values()

409     113.918693
1513    113.918693
1511    113.918693
1509    113.918693
1782    113.918693
           ...    
1295       114.282
1842    114.321336
997      22.280605
975      22.280605
1024     22.280605
Name: lng, Length: 945, dtype: object